In [35]:
import pandas as pd

PERFORMER_CSV = r'D:\repo\Stock\Seasonal-Stock\dataset\seasonal_top_performer.csv'
df = pd.read_csv(PERFORMER_CSV)
df.head()

,month and week,top performer 1,top performer 2,top performer 3,top performer 1 gain/loss ratio,top performer 2 gain/loss ratio,top performer 3 gain/loss ratio,num years,month,week,max profit 2024,max profit 0,max profit 1,max profit 2
0,1-1,APHA,ACB,CAE,29.354626,16.995886,2.965446,6.0,1.0,1.0,1.000000,-0.025294,-0.012209,-0.014439
1,1-2,APHA,CSU,OTEX,19.151603,8.899261,2.562831,6.0,1.0,2.0,1.000000,-0.025294,-0.012209,-0.014439
2,1-3,SHOP,APHA,BNS,4.508222,2.988331,1.404278,6.0,1.0,3.0,-0.016109,-0.025294,-0.012209,-0.014439
3,1-4,APHA,ACB,SHOP,15.532673,14.333630,6.842593,6.0,1.0,4.0,1.000000,-0.025294,-0.012209,-0.014439
4,2-1,APHA,CAE,SHOP,24.394333,4.652913,3.896730,6.0,2.0,1.0,1.000000,-0.025294,-0.012209,-0.014439


In [36]:
import os
from datetime import datetime, timedelta
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from dotenv import load_dotenv

load_dotenv()
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_S_KEY = os.getenv("ALPACA_S_KEY")
stock_client = StockHistoricalDataClient(
    ALPACA_API_KEY, ALPACA_S_KEY, raw_data=True
)

def get_first_day_of_week(year, month, week):
    # Given the year, month, and week number, return the first day of that week
    # The week number is 1-based

    # Calculate the first day of the month
    first_day = datetime(year, month, 1)

    # Calculate the weekday of the first day of the month
    first_day_weekday = first_day.weekday()

    # Calculate the number of days to add to get to the first day of the week
    days_to_add = (week - 1) * 7 - first_day_weekday

    # Calculate the first day of the week
    first_day_of_week = first_day + timedelta(days=days_to_add)

    return first_day_of_week

def get_max_profit(sym, start_date, end_date, timeframe = TimeFrame.Day):
    # Get the maximum profit for a given stock symbol between the close of the start date and the max within the duration
    params = {
        "symbol_or_symbols": sym,
        "timeframe": timeframe,
        "start": start_date,
        "end": end_date,
    }
    request_params = StockBarsRequest(**params)
    res = stock_client.get_stock_bars(request_params)
    # print(request_params)
    # print(res)
    data = res[sym]
    if data[0] is None:
        return 1
    # print(data)
    start_price = data[0]["c"]    
    max_price = start_price
    max_profit = 1
    for entry in data:
        if entry['h'] > max_price:
            max_price = entry['h']
            max_profit = max(max_profit, (max_price - start_price) / start_price)
            
    if max_profit == 1:
        # simulate selling off the stock at the end of the week
        return (data[-1]['c'] - start_price) / start_price
    return max_profit
df[['max profit 0', 'max profit 1', 'max profit 2']] = 1
for index, row in df.iterrows():
    month_and_week = row["month and week"]
    # Perform operations on each month and week value
    print(month_and_week)
    # Extract the month and week number from the month_and_week variable
    month, week = map(int, month_and_week.split("-"))

    first_week_day = get_first_day_of_week(2024, month, week)
    # set start date to the day before the first day of the week
    start_date = first_week_day - timedelta(days=1)
    today = datetime.now()
    # set start date as first day of 2024
    # start_date = datetime(2023, 1, 1)
    # # set end date as today
    # end_date = today
    end_date = first_week_day + timedelta(days=6)
    if end_date > today - timedelta(days=3):
        break
    top_performer_1 = row["top performer 1"]
    top_performer_2 = row["top performer 2"]
    top_performer_3 = row["top performer 3"]
    for i, performer in enumerate((top_performer_1, top_performer_2, top_performer_3)):
        max_profit = get_max_profit(performer, start_date, end_date)
        print(f"Max profit for {performer} is {max_profit}")
        top_performer = performer
        # save the max profit at the corresponding row and column based on the month and week
        df.at[index, f"max profit {i}"] = max_profit
            
# save the df to a csv
df.to_csv(PERFORMER_CSV, index=False)

1-1
Max profit for APHA is 1
Max profit for ACB is 0.0004212299915753538
Max profit for CAE is 0.003764705882352861
1-2


C:\Users\Michael\AppData\Local\Temp\ipykernel_85024\3546185319.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0004212299915753538' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"max profit {i}"] = max_profit
C:\Users\Michael\AppData\Local\Temp\ipykernel_85024\3546185319.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.003764705882352861' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"max profit {i}"] = max_profit


Max profit for APHA is 1
Max profit for CSU is 1
Max profit for OTEX is 0.006296924194720223
1-3
Max profit for SHOP is -0.016109198229217758
Max profit for APHA is 1
Max profit for BNS is -0.004741379310344803
1-4


C:\Users\Michael\AppData\Local\Temp\ipykernel_85024\3546185319.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.016109198229217758' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"max profit {i}"] = max_profit


Max profit for APHA is 1
Max profit for ACB is -0.0497619047619047
Max profit for SHOP is 0.016072763518564563
2-1
Max profit for APHA is 1
Max profit for CAE is -0.031019202363367752
Max profit for SHOP is -0.007301891309552303
2-2
Max profit for SU is 0.00910804020100511
Max profit for BTO is -0.006624825662482611
Max profit for CVE is 0.031130876747141056
2-3
Max profit for CSU is 1
Max profit for SU is 0.021624961383997617
Max profit for CAE is -0.11416589002795896
2-4
Max profit for WEED is 0.0753328997452886
Max profit for SHOP is -0.030272195370134945
Max profit for IMO is 0.04317643074839141
3-1
Max profit for SHOP is -0.012506446621970072
Max profit for OTEX is 0.020800832033281258
Max profit for RY is -0.006532612024089013
3-2
Max profit for APHA is 1
Max profit for ACB is 0.049504950495049625
Max profit for SHOP is 0.005147155866437912
3-3
Max profit for APHA is 1
Max profit for ACB is 0.052980132450331174
Max profit for SHOP is 0.029078297985861104
3-4
Max profit for SU is 

Result indicates no verifable seasonal trend with highlighted stocks
For future reference, stock selection should be validated prior to data training to prevent stocks like APHA which has been discontinued in 2021 from showing up in the results